In [1]:
import os
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/intent_classification/research


In [2]:
# Goto Root directory of the project
os.chdir("../")
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/intent_classification


In [3]:
# Data Ingestion Config Schema/Data Class
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path
    local_train_csv_path: Path
    local_test_csv_path: Path

In [4]:
from src.textClassifier.constants import *
from src.textClassifier.utils.common import read_yaml, create_directories
from box import ConfigBox

## Config Manager

In [5]:
# Create Configuration Manager: This has basic information required before staring any module/components

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config:ConfigBox = read_yaml(config_path)
        self.params:ConfigBox = read_yaml(params_path)

        # Create Artifacts Root dir
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name,
            local_train_csv_path= config.local_train_csv_path,
            local_test_csv_path= config.local_test_csv_path
        ) 

        return data_transformation_config


In [6]:
## Data Transfomation operation libraries import

import os
from src.textClassifier.logging import logger
from transformers import DistilBertTokenizer
from datasets import load_dataset

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/intent_classification/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-10 23:07:14,918 : INFO: config: PyTorch version 2.7.0 available.]


## Data Transformation Component

In [7]:
class DataTransformation:
    def __init__(self, data_transformation_config: DataTransformationConfig):
        self.config = data_transformation_config

        self.tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

        self.label2id = {
            "Intent_Lease_Abstraction": 0,
            "Intent_Comparison_LOI_Lease": 1,
            "Intent_Clause_Protect": 2,
            "Intent_Company_research": 3,
            "Intent_Transaction_Date_navigator": 4,
            "Intent_Amendment_Abstraction": 5,
            "Intent_Sales_Listings_Comparison": 6,
            "Intent_Lease_Listings_Comparison": 7,
        }

        self.id2label = {v: k for k, v in self.label2id.items()}

    # Preprocessing function with label mapping
    def preprocess_function(self,examples):
        # Tokenize the text field
        tokenized = self.tokenizer(examples['email_text'], padding="max_length", truncation=True)

        # Convert class labels to integers
        tokenized["labels"] = [self.label2id[label] for label in examples["intent"]]
        return tokenized


    def apply_transformation_to_dataset(self):
        # Load local CSV file
        raw_dataset = load_dataset(
            'csv',
            data_files={
                'train': self.config.local_train_csv_path,
                'validation': self.config.local_test_csv_path
            }
        )


        # Apply preprocessing
        encoded_dataset = raw_dataset.map(self.preprocess_function, batched=True)

        encoded_dataset.save_to_disk(os.path.join(self.config.root_dir, "intent-dataset"))


In [8]:
# Instantiate and call the data transformation methods
config = ConfigurationManager()

data_transformation_config = config.get_data_transformation_config()

data_transformation = DataTransformation(data_transformation_config)

data_transformation.apply_transformation_to_dataset()

[2025-05-10 23:07:15,133 : INFO: common: yaml file: config/config.yaml is loaded sucessfully]
[2025-05-10 23:07:15,135 : INFO: common: yaml file: params.yaml is loaded sucessfully]
[2025-05-10 23:07:15,135 : INFO: common: Created directory at: artifacts]
[2025-05-10 23:07:15,136 : INFO: common: Created directory at: artifacts/data_transformation]


Generating train split: 8000 examples [00:00, 515065.12 examples/s]
Generating validation split: 800 examples [00:00, 290162.85 examples/s]
Saving the dataset (1/1 shards): 100%|██████████| 800/800 [00:00<00:00, 172339.15 examples/s]
